In [ ]:
#配置参数
import argparse
parser = argparse.ArgumentParser()#创建一个对象 下面是设置各种参数
parser.add_argument("--epochs", type=int, default=200, help="number of epochs of training")
parser.add_argument("--batch_size", type=int, default=64, help="size of the batches")
parser.add_argument("--lr", type=float, default=0.0002, help="learning rate")
opt = parser.parse_args()

In [ ]:
#面部图片Dataset
import os
from PIL import Image
from torch.utils.data import Dataset,DataLoader
class Face(Dataset):
    def __init__(self, root, transforms=None):
        imgs = os.listdir(root) #这里存的不是所有图片，是图片的路径，在调用__getitem__的时候才会加载图片
        self.imgs = [os.path.join(root,img) for img in imgs] #路径拼接

        self.transforms = transforms #对图片进行一些调整操作

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, idx):
        img_path = self.imgs[idx] #根据路径返回图片
        data = Image.open(img_path)
        if self.transforms:
            data=self.transforms(data)
        return data

In [ ]:
#生成器网络，接收随机噪声作为输入，生成逼真的图像。
import torch.nn as nn
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, z):
        pass

In [ ]:
ndf = 64 #通道数，这里设置的有点大有待研究
#判别器网络，尝试区分真实的图像和生成器生成的图像
class Discriminator():
    def __init__(self):
        super().__init__()

        #输入大小为3 * 64 * 64
        self.model = nn.Sequential( #Sequential是一个模块容器，用了这个就不用再写一遍向前传播了 
            nn.Conv2d(3, ndf, 4, 2, 1, bias=False), #前5个参数分别是输入通道数，输出通道数，卷积核大小，卷积核每次移动多少个像素，原图片边缘加几个空白像素
            nn.LeakyReLU(0.2, inplace=True), #激活函数,inplace=True直接修改原数据
            #输出大小为 (ndf) * 32 * 32 。这里有个计算公式 输出尺寸 = (输入尺寸 + 2*填充 - 卷积核大小) / 步长 + 1，除法是向下取整

            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2), #百度的：进行数据归一化处理，使数据在Relu之前不会因为数据过大导致网络性能不稳定
            nn.LeakyReLU(0.2, inplace=True),
            #输出大小为 (ndf*2) * 16 * 16

            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2), 
            nn.LeakyReLU(0.2, inplace=True),
            #输出大小为  (ndf*4) * 8 * 8

            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            #输出大小为 (ndf*8) * 4 * 4

            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid() #激活函数，输出0到1之间的概率
        )
    def forward(self, img):
        return self.model(img)

In [ ]:
#加载数据集
FaceData = Face('./img_align_celeba', transforms=tfm)
data_loader = DataLoader(FaceData,batch_size=opt.batch_size,shuffle=True)#数据读取

In [ ]:
import torch
if __name__ == '__main__':
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")#设备
    print('当前device为' + str(device))

    generator = Generator().to(device) #生成器
    discriminator = Discriminator().to(device) #判别器
    adversarial_loss = torch.nn.BCELoss().to(device) #损失函数

    optimizer_G = torch.optim.Adam(generator.parameters(), lr=opt.lr ) #优化器
    optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=opt.lr ) #优化器

    os.makedirs("generated-facial-images", exist_ok=True) #创建存储生成图片的目录，exist_ok=True就可以让文件夹存在的时候也不会报错

    #开始训练
    for epoch in range(opt.epochs):
        for imgs in data_loader:
            pass